Combine IFL Rosters on Sleeper, LateRound and Fantasy Calc Rankings

In [1]:
import polars as pl
pl.Config.set_tbl_rows(-1)
rel_path_roster = '../data/2025/ifl/sleeper-rosters.csv'
rel_path_fc = '../data/2025/fantasy_calc/false_2_10_ppr_1.csv'
rel_path_lr = '../data/2025/late_round/superflex-rankings.csv'
rel_path_rookies = '../data/2025/ifl/rookies-personal-rank.csv'

from preprocessing import *
from rookies import merge_rookies

late_round = preprocess_late_round(rel_path_lr)
fantasy_calc = preprocess_fantasy_calc(rel_path_fc)
df = create_auction_values(late_round, fantasy_calc)
df = df.sort('auction_late_round', descending=True)

rookies = merge_rookies(rel_path_rookies, df)
rookies = rookies.filter(pl.col('personal_rank').is_not_null()).sort('personal_rank', descending=False)
rookies.write_csv('../data/2025/big_boards/rookie-bb-ifl.csv')

ifl = pl.read_csv(rel_path_roster).drop(['headshot', 'nickname', 'roster_slot', 'position', 'full_name'])
df_ifl = df.join(ifl, left_on='sleeper_id', right_on='player_id', how='left')
df_ifl = df_ifl.sort('auction_late_round', descending=True)
df_ifl.write_csv('../data/2025/big_boards/auction-bb-ifl.csv')

Free Agency/Rostered Value

In [6]:
import polars as pl
team_values = (df_ifl.group_by("team_name").agg(pl.col("redraft_value").sum().alias("redraft_value")).sort("redraft_value", descending=True))

free_agent_value = team_values.filter(pl.col("team_name").is_null())["redraft_value"].sum()
rostered_value = team_values.filter(pl.col("team_name").is_not_null())["redraft_value"].sum()

print(f"Free Agent Value: {free_agent_value}")
print(f"Rostered Value: {rostered_value}")
print(f"Percent Rostered: {round(rostered_value / (rostered_value + free_agent_value), 2)}")

Free Agent Value: 284851
Rostered Value: 145066
Percent Rostered: 0.34


Drafts - 2024

In [2]:
import os, sys
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
import polars as pl 
pl.Config.set_tbl_rows(-1)

from ifl_drafts import get_ifl_draft_metadata
df = get_ifl_draft_metadata()

drafted = df.filter(pl.col('drafted') == True).drop(['price'])
auctioned = df.filter(pl.col('drafted') == False)

SchemaError: type Boolean is incompatible with expected type Null

In [5]:
contracts = pl.read_csv('../data/2025/Contracts-IFL.csv')
contracts.filter(pl.col('season') == 2024).sort('price', descending=True)
contracts = contracts.drop(['player_id', 'round', 'pick', 'drafted', 'user_id', 'display_name'])
drafted = drafted.drop(['years'])
drafted_prices = drafted.join(contracts, on = ['season', 'player_name'], how = 'left').head(5)

In [6]:
print(auctioned.columns)
print(drafted_prices.columns)
combined = pl.concat([auctioned, drafted_prices])

['season', 'player_id', 'player_name', 'round', 'pick', 'user_id', 'team_name', 'display_name', 'price', 'drafted', 'years']
['season', 'player_id', 'player_name', 'round', 'pick', 'user_id', 'team_name', 'display_name', 'drafted', 'price', 'years']


ShapeError: unable to vstack, column names don't match: "price" and "drafted"